# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Clash Royale Predictions** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, generate PNG for forecast
3. Store predictions in a monitoring feature group adn generate PNG for hindcast

In [26]:
import hopsworks
import datetime
import pandas as pd
from xgboost import Booster, DMatrix  # Import DMatrix for predictions
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import json
import os
import requests

with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()
fs = project.get_feature_store() 

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="clash_royale_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()
saved_model_dir

2025-01-08 02:14:23,323 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 02:14:23,334 INFO: Initializing external client
2025-01-08 02:14:23,334 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 02:14:25,085 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700


'C:\\Users\\steve\\AppData\\Local\\Temp\\a1e98b90-b9e0-4ffc-a2b6-5da632d8e9df\\clash_royale_xgboost_model/1'

In [27]:
# Replace this with your Clash Royale API token
key_file_path = os.path.join('../data/clash-royale-api-key.txt')
with open(key_file_path, 'r') as file:
    api_key = file.read().strip()

API_TOKEN = api_key
PLAYER_TAG = "#2LGY9G"  # Replace with the player's tag (include the #)

# Headers for the API request
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}"
}

# URL for player information
BASE_URL = "https://api.clashroyale.com/v1"
PLAYER_URL = f"{BASE_URL}/players/{PLAYER_TAG.replace('#', '%23')}"
BATTLE_LOG_URL = f"{PLAYER_URL}/battlelog"

MODEL_PATH = os.path.join("clash_royale_model", "model.json")

# Card ID mapping
card_numbers = {
    "Archers": 1, "Archer Queen": 2, "Baby Dragon": 3, "Balloon": 4, "Bandit": 5, "Barbarians": 6,
    "Bats": 7, "Battle Healer": 8, "Battle Ram": 9, "Bomber": 10, "Bowler": 11, "Bush Goblins": 12,
    "Cannon Cart": 13, "Cursed Hog": 14, "Dark Prince": 15, "Dart Goblin": 16, "Electro Dragon": 17,
    "Electro Giant": 18, "Electro Spirit": 19, "Electro Wizard": 20, "Elite Barbarians": 21,
    "Elixir Blob": 22, "Elixir Golem": 23, "Elixir Golemite": 24, "Executioner": 25, "Firecracker": 26,
    "Fire Spirit": 27, "Fisherman": 28, "Flying Machine": 29, "Giant": 30, "Giant Skeleton": 31,
    "Goblin Brawler": 32, "Goblin Gang": 33, "Goblin Demolisher": 34, "Goblin Giant": 35,
    "Goblin Machine": 36, "Goblins": 37, "Goblinstein": 38, "Golden Knight": 39, "Golem": 40,
    "Golemite": 41, "Guardienne": 42, "Guards": 43, "Hog Rider": 44, "Hunter": 45, "Heal Spirit": 46,
    "Ice Golem": 47, "Ice Spirit": 48, "Ice Wizard": 49, "Inferno Dragon": 50, "Knight": 51,
    "Lava Hound": 52, "Lava Pup": 53, "Little Prince": 54, "Lumberjack": 55, "Magic Archer": 56,
    "Mega Knight": 57, "Mega Minion": 58, "Mighty Miner": 59, "Miner": 60, "Mini P.E.K.K.A.": 61,
    "Minion Horde": 62, "Minions": 63, "Monk": 64, "Mother Witch": 65, "Monster": 66, "Musketeer": 67,
    "Night Witch": 68, "P.E.K.K.A.": 69, "Phoenix": 70, "Reborn Phoenix": 71, "Prince": 72,
    "Princess": 73, "Ram Rider": 74, "Rascal Boy": 75, "Rascal Girl": 76, "Royal Ghost": 77,
    "Royal Giant": 78, "Royal Hogs": 79, "Royal Recruits": 80, "Skeleton Army": 81,
    "Skeleton Barrel": 82, "Skeleton Dragons": 83, "Skeleton King": 84, "Skeletons": 85, "Sparky": 86,
    "Spear Goblins": 87, "Suspicious Bush": 88, "Three Musketeers": 89, "Valkyrie": 90,
    "Wall Breakers": 91, "Witch": 92, "Wizard": 93, "Zappies": 94,"Bomb Tower": 95, "Cannon": 96, "Cannon Cart (broken)": 97, "Inferno Tower": 98, "Mortar": 99,
        "Tesla": 100, "X-Bow": 101,"Barbarian Hut": 102, "Elixir Collector": 103, "Furnace": 104, "Goblin Cage": 105,
        "Goblin Drill": 106, "Goblin Hut": 107, "Phoenix Egg": 108, "Tombstone": 109, "Arrows": 110, "Barbarian Barrel": 111, "Earthquake": 112, "Fireball": 113, "Freeze": 114,
        "Giant Snowball": 115, "Goblin Curse": 116, "Lightning": 117, "Poison": 118, "Rage": 119, "Rocket": 120, "Royal Delivery": 121, "The Log": 122, "Tornado": 123, "Void": 124, "Zap": 125,
        "Barbarian Barrel": 126, "Barbarian Hut": 127, "Battle Ram": 128, "Elixir Golem": 129,
        "Elixir Golemite": 130, "Furnace": 131, "Goblin Barrel": 132, "Goblin Cage": 133, "Goblin Curse": 134,
        "Goblin Drill": 135, "Goblin Giant": 136, "Goblin Hut": 137, "Golem": 138, "Graveyard": 139,
        "Lava Hound": 140, "Little Prince": 141, "Mother Witch": 142, "Night Witch": 143,
        "Phoenix Egg": 144, "Royal Delivery": 145, "Skeleton Barrel": 146, "Skeleton King": 147,
        "Suspicious Bush": 148, "Tombstone": 149, "Witch": 150,
        "Archers/Evolution": 155, "Barbarians/Evolution": 156, "Battle Ram/Evolution": 157,
        "Bats/Evolution": 158, "Bomber/Evolution": 159, "Cannon/Evolution": 160,
        "Electro Dragon/Evolution": 161, "Firecracker/Evolution": 162, "Giant Snowball/Evolution": 163,
        "Goblin Barrel/Evolution": 164, "Goblin Cage/Evolution": 165, "Goblin Drill/Evolution": 166,
        "Goblin Giant/Evolution": 167, "Ice Spirit/Evolution": 168, "Knight/Evolution": 169,
        "Mega Knight/Evolution": 170, "Mortar/Evolution": 171, "Musketeer/Evolution": 172,
        "P.E.K.K.A/Evolution": 173, "Royal Giant/Evolution": 174, "Royal Recruits/Evolution": 175,
        "Skeletons/Evolution": 176, "Tesla/Evolution": 177, "Valkyrie/Evolution": 178,
        "Wall Breakers/Evolution": 179, "Wizard/Evolution": 180, "Zap/Evolution": 181

}

def fetch_battle_log():
    """Fetch the battle log of the player."""
    try:
        response = requests.get(BATTLE_LOG_URL, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching battle log: {e}")
        return None


def deck_to_ids(deck, mapping):
    """Convert card names in the deck to IDs based on mapping."""
    return [mapping.get(card["name"], 0) - 1 for card in deck]  # Subtract 1 for zero-based indexing


def preprocess_game_data(game_data):
    """Prepare game data for prediction."""
    team_deck_ids = deck_to_ids(game_data["team"][0]["cards"], card_numbers)

    # One-hot encode the deck
    num_choices = 181  # Total number of card IDs
    one_hot = np.zeros(num_choices, dtype=int)
    one_hot[np.array(team_deck_ids)] = 1  # Set indices corresponding to cards to 1

    # Create additional features
    trophy_difference = game_data["team"][0].get("trophies", 0) - game_data["opponent"][0].get("trophies", 0)
    elixir_leaked = game_data["team"][0].get("elixirLeaked", 0)

    # Combine features
    features = np.concatenate(([trophy_difference, elixir_leaked], one_hot))
    return pd.DataFrame([features])


def load_model(model_path):
    """Load the saved XGBoost model from JSON."""
    model = Booster()
    model.load_model(model_path)
    return model


def determine_outcome(last_game):
    """Determine the outcome of the last game."""
    team_crowns = last_game["team"][0].get("crowns", 0)
    opponent_crowns = last_game["opponent"][0].get("crowns", 0)

    if team_crowns > opponent_crowns:
        return "Victory"
    elif team_crowns < opponent_crowns:
        return "Defeat"
    else:
        return "Draw"

def main():
    # Fetch the last game data
    battle_log = fetch_battle_log()
    if not battle_log or len(battle_log) == 0:
        print("No battle log available.")
        return

    last_game = battle_log[0]  # Most recent game
    print("Last game retrieved:", last_game)


    # Preprocess the game data
    game_data = preprocess_game_data(last_game)
    print("Processed game data for prediction:", game_data)

    # Load the trained model
    model = load_model(MODEL_PATH)

    
    # Determine the outcome of the last game
    outcome = determine_outcome(last_game)
    print(f"Outcome of the last game: {outcome}")

    # Make predictions
    dmatrix = DMatrix(game_data)  # Convert data to DMatrix format
    prediction = model.predict(dmatrix)
    print(f"Prediction for the last game: {prediction}")


if __name__ == "__main__":
    main()


Last game retrieved: {'type': 'pathOfLegend', 'battleTime': '20250107T144347.000Z', 'isLadderTournament': False, 'arena': {'id': 54000103, 'name': 'Legendary Arena'}, 'gameMode': {'id': 72000450, 'name': 'Ranked1v1_NewArena'}, 'deckSelection': 'collection', 'team': [{'tag': '#2LGY9G', 'name': 'Diwel26', 'crowns': 1, 'kingTowerHitPoints': 855, 'princessTowersHitPoints': None, 'clan': {'tag': '#YULCL0YQ', 'name': 'ITA', 'badgeId': 16000081}, 'cards': [{'name': 'Royal Giant', 'id': 26000024, 'level': 11, 'starLevel': 2, 'evolutionLevel': 1, 'maxLevel': 14, 'maxEvolutionLevel': 1, 'rarity': 'common', 'elixirCost': 6, 'iconUrls': {'medium': 'https://api-assets.clashroyale.com/cards/300/mnlRaNtmfpQx2e6mp70sLd0ND-pKPF70Cf87_agEKg4.png', 'evolutionMedium': 'https://api-assets.clashroyale.com/cardevolutions/300/mnlRaNtmfpQx2e6mp70sLd0ND-pKPF70Cf87_agEKg4.png'}}, {'name': 'Firecracker', 'id': 26000064, 'level': 11, 'starLevel': 1, 'evolutionLevel': 1, 'maxLevel': 14, 'maxEvolutionLevel': 1, 'rar

In [29]:
import pandas as pd
from xgboost import Booster, DMatrix
import numpy as np


MODEL_PATH = "clash_royale_model/model.json"

def deck_to_ids(deck, mapping):
    """Converti i nomi delle carte in ID in base alla mappatura."""
    return [mapping.get(card, 0) - 1 for card in deck]  # Indici a base zero

def preprocess_deck(deck):
    """Prepara il deck scelto per il modello."""
    # Converti le carte in ID
    deck_ids = deck_to_ids(deck, card_numbers)

    # Effettua l'one-hot encoding
    num_choices = 181  # Numero totale di carte
    one_hot = np.zeros(num_choices, dtype=int)
    one_hot[np.array(deck_ids)] = 1  # Imposta 1 per le carte presenti

    # Aggiungi eventuali feature aggiuntive (placeholder per ora)
    trophy_difference = 0  # Placeholder per differenza trofei
    elixir_leaked = 0  # Placeholder per elisir sprecato

    # Combina le caratteristiche
    features = np.concatenate(([trophy_difference, elixir_leaked], one_hot))
    return pd.DataFrame([features])

def load_model(model_path):
    """Carica il modello XGBoost salvato."""
    model = Booster()
    model.load_model(model_path)
    return model


def main():
    # Richiedi all'utente di inserire le carte per il deck 2
    while True:
        print("Inserisci 8 carte per il tuo deck (separate da virgole):")
        user_input = input("Carte: ").strip()

        # Converti l'input in una lista di carte
        user_deck = [card.strip() for card in user_input.split(",")]

        if len(user_deck) != 8:
            print("Errore: Devi inserire esattamente 8 carte. Riprova.")
        else:
            break

    # Prepara i dati del deck per la previsione
    deck_data = preprocess_deck(user_deck)
    print("Dati pre-elaborati per il modello:")
    print(deck_data)

    # Carica il modello
    model = load_model(MODEL_PATH)

    # Effettua la previsione
    dmatrix = DMatrix(deck_data)  # Converti i dati in formato DMatrix
    prediction = model.predict(dmatrix)
    print(f"Risultato della previsione per il deck scelto: {prediction}")

if __name__ == "__main__":
    main()

Inserisci 8 carte per il tuo deck (separate da virgole):
Dati pre-elaborati per il modello:
   0    1    2    3    4    5    6    7    8    9    ...  173  174  175  176  \
0    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   177  178  179  180  181  182  
0    0    0    0    0    0    0  

[1 rows x 183 columns]
Risultato della previsione per il deck scelto: [0.4697614]


In [31]:
import gradio as gr
import pandas as pd
from xgboost import Booster, DMatrix
import numpy as np


MODEL_PATH = "clash_royale_model/model.json"

def deck_to_ids(deck, mapping):
    """Converti i nomi delle carte in ID in base alla mappatura."""
    return [mapping.get(card, 0) - 1 for card in deck]  # Indici a base zero

def preprocess_deck(deck):
    """Prepara il deck scelto per il modello."""
    # Converti le carte in ID
    deck_ids = deck_to_ids(deck, card_numbers)

    # Effettua l'one-hot encoding
    num_choices = 181  # Numero totale di carte
    one_hot = np.zeros(num_choices, dtype=int)
    one_hot[np.array(deck_ids)] = 1  # Imposta 1 per le carte presenti

    # Aggiungi eventuali feature aggiuntive (placeholder per ora)
    trophy_difference = 0  # Placeholder per differenza trofei
    elixir_leaked = 0  # Placeholder per elisir sprecato

    # Combina le caratteristiche
    features = np.concatenate(([trophy_difference, elixir_leaked], one_hot))
    return pd.DataFrame([features])

def load_model(model_path):
    """Carica il modello XGBoost salvato."""
    model = Booster()
    model.load_model(model_path)
    return model

# Carica il modello all'avvio
model = load_model(MODEL_PATH)

def predict_outcome(opponent_deck):
    """Effettua una previsione basata sul deck dell'avversario."""
    # Prepara i dati del deck dell'avversario
    deck_data = preprocess_deck(opponent_deck)

    # Effettua la previsione
    dmatrix = DMatrix(deck_data)  # Converti i dati in formato DMatrix
    prediction = model.predict(dmatrix)

    # Interpreta la previsione
    result = f"Probabilità di vincere: {prediction[0] * 100:.2f}%"
    return result

# Lista di carte per la selezione
card_list = list(card_numbers.keys())

# Creazione dell'interfaccia Gradio
with gr.Blocks() as interface:
    gr.Markdown("## Clash Royale Prediction")
    gr.Markdown("Seleziona le 8 carte del deck dell'avversario per vedere la probabilità di vincere!")
    
    opponent_deck = gr.CheckboxGroup(
        choices=card_list,
        label="Seleziona 8 carte dell'avversario:",
        info="Scegli esattamente 8 carte."
    )
    
    result = gr.Textbox(label="Risultato della previsione:", interactive=False)

    def validate_and_predict(deck):
        """Verifica se l'utente ha scelto 8 carte e calcola la previsione."""
        if len(deck) != 8:
            return "Errore: seleziona esattamente 8 carte."
        return predict_outcome(deck)
    
    predict_button = gr.Button("Effettua la previsione")
    predict_button.click(validate_and_predict, inputs=[opponent_deck], outputs=[result])

# Avvia l'interfaccia
interface.launch()

* Running on local URL:  http://127.0.0.1:7861
2025-01-08 02:26:50,489 INFO: HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-01-08 02:26:50,503 INFO: HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


2025-01-08 02:26:50,839 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [32]:
# ULTIMA PROVA UN PO STRANA "NON GUARDARE"
MODEL_PATH = "clash_royale_model/model.json"

# Il tuo deck personale (rimane fisso)
my_deck = ["Royal Giant", "Firecracker", "Fireball", "Knight", "Goblin Barrel", "Tombstone", "Fisherman", "Princess"]

# Deck predefiniti
predefined_decks = {
    "Aggro Deck": ["Royal Giant", "Firecracker", "Fireball", "Knight", "Goblin Barrel", "Tombstone", "Fisherman", "Princess"],
    "Control Deck": ["Archer Queen", "Barbarian Barrel", "Fireball", "Knight", "Fisherman", "Tombstone", "Princess", "Baby Dragon"],
    "Balanced Deck": ["Knight", "Princess", "Royal Giant", "Goblin Barrel", "Fireball", "Fisherman", "Tombstone", "Firecracker"]
}

def deck_to_ids(deck, mapping):
    """Converti i nomi delle carte in ID in base alla mappatura."""
    return [mapping.get(card, 0) - 1 for card in deck]  # Indici a base zero

def preprocess_deck(deck):
    """Prepara il deck scelto per il modello."""
    # Converti le carte in ID
    deck_ids = deck_to_ids(deck, card_numbers)

    # Effettua l'one-hot encoding
    num_choices = 181  # Numero totale di carte
    one_hot = np.zeros(num_choices, dtype=int)
    one_hot[np.array(deck_ids)] = 1  # Imposta 1 per le carte presenti

    # Aggiungi eventuali feature aggiuntive (placeholder per ora)
    trophy_difference = 0  # Placeholder per differenza trofei
    elixir_leaked = 0  # Placeholder per elisir sprecato

    # Combina le caratteristiche
    features = np.concatenate(([trophy_difference, elixir_leaked], one_hot))
    return pd.DataFrame([features])

def load_model(model_path):
    """Carica il modello XGBoost salvato."""
    model = Booster()
    model.load_model(model_path)
    return model

# Carica il modello all'avvio
model = load_model(MODEL_PATH)

def predict_outcome(deck):
    """Effettua una previsione basata sul deck."""
    # Prepara i dati del deck
    deck_data = preprocess_deck(deck)

    # Effettua la previsione
    dmatrix = DMatrix(deck_data)  # Converti i dati in formato DMatrix
    prediction = model.predict(dmatrix)

    # Interpreta la previsione
    return prediction[0] * 100

def predict_both_decks(opponent_deck_name):
    """Effettua previsioni per il deck personale e quello dell'avversario."""
    if opponent_deck_name not in predefined_decks:
        return "Errore: deck avversario non valido."

    opponent_deck = predefined_decks[opponent_deck_name]

    # Previsione per il deck personale (fisso)
    my_deck_prediction = predict_outcome(my_deck)

    # Previsione per il deck dell'avversario
    opponent_deck_prediction = predict_outcome(opponent_deck)

    return f"Probabilità di vincere con il tuo deck: {my_deck_prediction:.2f}%\n" \
           f"Probabilità di vincere contro il {opponent_deck_name}: {opponent_deck_prediction:.2f}%"

# Creazione dell'interfaccia Gradio
with gr.Blocks() as interface:
    gr.Markdown("## Clash Royale Prediction")
    gr.Markdown("Il tuo deck personale rimane fisso. Seleziona il deck dell'avversario per vedere le previsioni!")

    # Visualizza il deck personale
    gr.Markdown(f"### Il tuo deck personale: {', '.join(my_deck)}")

    # Selezione del deck dell'avversario
    opponent_deck_dropdown = gr.Dropdown(
        choices=list(predefined_decks.keys()),
        label="Seleziona un deck dell'avversario:",
    )

    result = gr.Textbox(label="Risultato della previsione:", interactive=False)

    predict_button = gr.Button("Effettua la previsione")
    predict_button.click(
        predict_both_decks,
        inputs=[opponent_deck_dropdown],
        outputs=[result]
    )

# Avvia l'interfaccia
interface.launch()

* Running on local URL:  http://127.0.0.1:7862
2025-01-08 02:32:48,619 INFO: HTTP Request: GET http://127.0.0.1:7862/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-01-08 02:32:48,630 INFO: HTTP Request: HEAD http://127.0.0.1:7862/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


2025-01-08 02:32:49,072 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
